# feature engineering 
## 시계열 
11. 카테고리 별 시간 간격
12. 누적판매량/누적주문량/누적방송횟수(카테고리(~ 1개월전,~ 2개월전,~6개월전, 없으면 None,&연속)
13. 조기매진 시간
14. 결합상품(0 : 3+4 ,1 : 다른상품이 묶이면 or +) 
15. 추세
16. 가격별 카테고리 (6,5000미만 : 저가, 6,5000~20만원 : 중저가 , 20만 이상 : 고가) → 50000빼고 다시보기
17. 주문량
18. 할인률(예진언니 참고자료 참고) 
    [Predict Future Sales] playground 커널 리뷰 1 - [https://dailyheumsi.tistory.com/m/109](https://dailyheumsi.tistory.com/m/109)

In [1]:
%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.style.use('seaborn') 
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

In [79]:
import datetime 

In [4]:
original = pd.read_csv("2019실적데이터_v1_200818.csv")
meta = pd.read_csv('meta_v02_0819.csv')
sales = pd.read_csv('sale_data_v02_0819.csv')

In [5]:
sales.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0


In [6]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37372 entries, 0 to 37371
Data columns (total 14 columns):
방송일시      37372 non-null object
노출(분)     20588 non-null float64
마더코드      37372 non-null int64
상품코드      37372 non-null int64
상품명       37372 non-null object
상품군       37372 non-null object
판매단가      37372 non-null int64
취급액       35379 non-null float64
NEW상품명    37372 non-null object
결제방법      37372 non-null int64
단위        22066 non-null object
브랜드       33066 non-null object
옵션        4140 non-null object
종류        22744 non-null object
dtypes: float64(2), int64(4), object(8)
memory usage: 4.0+ MB


In [9]:
sales.loc[sales["취급액"].isna()] # 1993개 = 주문량 0  

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류
144,2019-01-02 22:00:00,NaN,100148,200432,무이자 LG 울트라HD TV 55UK6800HNC,가전,1440000,NaN,LG 울트라HD TV 55인치,2,55인치,LG,NaN,TV
147,2019-01-02 22:00:00,NaN,100148,200518,일시불 LG 울트라HD TV 70UK7400KNA,가전,2700000,NaN,LG 울트라HD TV 70인치,1,70인치,LG,NaN,TV
148,2019-01-02 22:00:00,NaN,100148,200451,무이자 LG 울트라HD TV 70UK7400KNA,가전,2990000,NaN,LG 울트라HD TV 70인치,2,70인치,LG,NaN,TV
153,2019-01-02 22:20:00,NaN,100148,200518,일시불 LG 울트라HD TV 70UK7400KNA,가전,2700000,NaN,LG 울트라HD TV 70인치,1,70인치,LG,NaN,TV
154,2019-01-02 22:20:00,NaN,100148,200451,무이자 LG 울트라HD TV 70UK7400KNA,가전,2990000,NaN,LG 울트라HD TV 70인치,2,70인치,LG,NaN,TV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36815,2019-12-25 10:20:00,NaN,100036,200070,구찌 인터로킹 GG 탑핸들 체인 숄더 스몰,잡화,2590000,NaN,구찌 인터로킹 GG 탑핸들 체인 숄더 스몰,0,스몰,구찌,NaN,가방
37059,2019-12-28 10:20:00,NaN,100036,200070,구찌 인터로킹 GG 탑핸들 체인 숄더 스몰,잡화,2590000,NaN,구찌 인터로킹 GG 탑핸들 체인 숄더 스몰,0,스몰,구찌,NaN,가방
37061,2019-12-28 10:20:00,NaN,100039,200073,버버리 홀스페리 페이톤 크로스백,잡화,880000,NaN,버버리 홀스페리 페이톤 크로스백,0,NaN,버버리,NaN,가방
37111,2019-12-28 21:20:00,NaN,100372,201169,(싱글+싱글)일월 품안애 온수매트,생활용품,198000,NaN,일월 품안애 온수매트 싱글 + 싱글,0,싱글+싱글,일월,NaN,NaN


In [10]:
sales["취급액"] = sales["취급액"].fillna(0)
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37372 entries, 0 to 37371
Data columns (total 14 columns):
방송일시      37372 non-null object
노출(분)     20588 non-null float64
마더코드      37372 non-null int64
상품코드      37372 non-null int64
상품명       37372 non-null object
상품군       37372 non-null object
판매단가      37372 non-null int64
취급액       37372 non-null float64
NEW상품명    37372 non-null object
결제방법      37372 non-null int64
단위        22066 non-null object
브랜드       33066 non-null object
옵션        4140 non-null object
종류        22744 non-null object
dtypes: float64(2), int64(4), object(8)
memory usage: 4.0+ MB


In [11]:
sales['방송일시'] = pd.to_datetime(sales['방송일시'])

In [14]:
print(sales["상품군"].nunique())
print(sales["상품군"].unique())

11
['의류' '속옷' '주방' '농수축' '이미용' '가전' '생활용품' '건강기능' '잡화' '가구' '침구']


In [12]:
# 방송시간 앞의 값으로 채워준 데이터 
sales2 = sales.copy()
sales2["노출(분)"] = sales2["노출(분)"].fillna(method='ffill')

In [13]:
sales2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37372 entries, 0 to 37371
Data columns (total 14 columns):
방송일시      37372 non-null datetime64[ns]
노출(분)     37372 non-null float64
마더코드      37372 non-null int64
상품코드      37372 non-null int64
상품명       37372 non-null object
상품군       37372 non-null object
판매단가      37372 non-null int64
취급액       37372 non-null float64
NEW상품명    37372 non-null object
결제방법      37372 non-null int64
단위        22066 non-null object
브랜드       33066 non-null object
옵션        4140 non-null object
종류        22744 non-null object
dtypes: datetime64[ns](1), float64(2), int64(4), object(7)
memory usage: 4.0+ MB


## 시간차 

### 1. 카테고리 별 시간 간격

In [24]:
# 전체에 한번에 하는거 짜려 했는데 실패 ^_^ ... 
def category_timelag(category) : 
    new = pd.DataFrame()
    
    for c in category :         
        category_df = sales.loc[sales["상품군"] == c]
        category_df["카테고리시간차"] = category_df["방송일시"].diff()
        new.append(category_df)
        
    return new.sort_index() 

In [25]:
category_timelag(sales["상품군"].unique())

C:\Users\LG\venv\jbeen\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


""


In [85]:
# 그래서 카테고리 하나하나마다 다 적용해 줘야 함 ^_^ ;; 
def category_timelag(category) : 
    category_df = sales.loc[sales["상품군"] == category]
    category_df["카테고리시간차"] = category_df["방송일시"].diff()
    
    # category_df["카테고리시간차"] = category_df["카테고리시간차"].map(lambda x : NaN if x.dt.seconds == 0 else x) 이거 적용이 안됨 ㅜㅜ 
    
    
    # 00:00:00 column 바꿔주기 (동일 상품인데 0값으로 채워졌으므로)  
    category_df["카테고리시간차"] = category_df["카테고리시간차"].dt.seconds.replace(0, np.NaN)
    category_df["카테고리시간차"] = category_df["카테고리시간차"].fillna(method='ffill') 
    
    category_df["카테고리시간차"] = category_df["카테고리시간차"].fillna(0) # 2019-1-1 06:00 상품값 0으로 채워줌 
    category_df["카테고리시간차"] = category_df["카테고리시간차"].apply(lambda x : str(datetime.timedelta(seconds=x))) # '초' 단위로 되어 있어서 바꿔주기 
    
    return category_df

In [86]:
clothes = category_timelag("의류")
furniture = category_timelag("가구")
ns = category_timelag("농수축")

C:\Users\LG\venv\jbeen\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\LG\venv\jbeen\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\LG\venv\jbeen\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [87]:
clothes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4340 entries, 0 to 37332
Data columns (total 15 columns):
방송일시       4340 non-null datetime64[ns]
노출(분)      3240 non-null float64
마더코드       4340 non-null int64
상품코드       4340 non-null int64
상품명        4340 non-null object
상품군        4340 non-null object
판매단가       4340 non-null int64
취급액        4340 non-null float64
NEW상품명     4340 non-null object
결제방법       4340 non-null int64
단위         2717 non-null object
브랜드        4340 non-null object
옵션         975 non-null object
종류         4340 non-null object
카테고리시간차    4340 non-null object
dtypes: datetime64[ns](1), float64(2), int64(4), object(8)
memory usage: 542.5+ KB


In [88]:
clothes.head(10)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,카테고리시간차
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,0:00:00
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,0:00:00
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,0:20:00
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,0:20:00
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,0:20:00
5,2019-01-01 06:40:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,0:20:00
9,2019-01-01 08:00:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,16133000.0,CERINI by PAT 남성 소프트 기모 릴렉스 팬츠,0,NaN,CERINI by PAT,NaN,1,1:20:00
10,2019-01-01 08:20:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,30061000.0,CERINI by PAT 남성 소프트 기모 릴렉스 팬츠,0,NaN,CERINI by PAT,NaN,1,0:20:00
11,2019-01-01 08:40:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,53542000.0,CERINI by PAT 남성 소프트 기모 릴렉스 팬츠,0,NaN,CERINI by PAT,NaN,1,0:20:00
12,2019-01-01 09:00:00,20.0,100816,202404,보코 리버시블 무스탕,의류,79000,16075000.0,보코 리버시블 무스탕,0,NaN,보코,NaN,2,0:20:00


In [204]:
clothes.head(50)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,카테고리시간차
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,0:00:00
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,0:00:00
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,0:20:00
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,0:20:00
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,0:20:00
5,2019-01-01 06:40:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,0:20:00
9,2019-01-01 08:00:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,16133000.0,CERINI by PAT 남성 소프트 기모 릴렉스 팬츠,0,NaN,CERINI by PAT,NaN,1,1:20:00
10,2019-01-01 08:20:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,30061000.0,CERINI by PAT 남성 소프트 기모 릴렉스 팬츠,0,NaN,CERINI by PAT,NaN,1,0:20:00
11,2019-01-01 08:40:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,53542000.0,CERINI by PAT 남성 소프트 기모 릴렉스 팬츠,0,NaN,CERINI by PAT,NaN,1,0:20:00
12,2019-01-01 09:00:00,20.0,100816,202404,보코 리버시블 무스탕,의류,79000,16075000.0,보코 리버시블 무스탕,0,NaN,보코,NaN,2,0:20:00


### 2. 동일 상품 별 시간 간격 
* NEW상품명 기준 ! 

In [89]:
print(sales["상품명"].nunique(), " -> ", sales["NEW상품명"].nunique())

1692  ->  1364


In [102]:
def product_timelag(sales) :
    sales["동일상품시간차"] = sales.groupby(["NEW상품명"])["방송일시"].diff()
    return sales 

In [103]:
sales = product_timelag(sales)

In [105]:
sales.loc[sales["NEW상품명"] == "마르엘라로사티 에코무스탕 1종"]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,동일상품시간차
36694,2019-12-24 06:20:00,20.0,100356,201134,마르엘라로사티 에코무스탕1종,의류,79000,16895000.0,마르엘라로사티 에코무스탕 1종,0,1종,마르엘라로사티,NaN,2,NaT
36695,2019-12-24 06:40:00,20.0,100356,201134,마르엘라로사티 에코무스탕1종,의류,79000,25022000.0,마르엘라로사티 에코무스탕 1종,0,1종,마르엘라로사티,NaN,2,0 days 00:20:00
36696,2019-12-24 07:00:00,20.0,100356,201134,마르엘라로사티 에코무스탕1종,의류,79000,49874000.0,마르엘라로사티 에코무스탕 1종,0,1종,마르엘라로사티,NaN,2,0 days 00:20:00
37330,2019-12-31 18:20:00,20.0,100356,201134,마르엘라로사티 에코무스탕1종,의류,79000,31983000.0,마르엘라로사티 에코무스탕 1종,0,1종,마르엘라로사티,NaN,2,7 days 11:20:00
37331,2019-12-31 18:40:00,20.0,100356,201134,마르엘라로사티 에코무스탕1종,의류,79000,39434000.0,마르엘라로사티 에코무스탕 1종,0,1종,마르엘라로사티,NaN,2,0 days 00:20:00
37332,2019-12-31 19:00:00,20.0,100356,201134,마르엘라로사티 에코무스탕1종,의류,79000,129905000.0,마르엘라로사티 에코무스탕 1종,0,1종,마르엘라로사티,NaN,2,0 days 00:20:00


In [106]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37372 entries, 0 to 37371
Data columns (total 15 columns):
방송일시       37372 non-null datetime64[ns]
노출(분)      20588 non-null float64
마더코드       37372 non-null int64
상품코드       37372 non-null int64
상품명        37372 non-null object
상품군        37372 non-null object
판매단가       37372 non-null int64
취급액        37372 non-null float64
NEW상품명     37372 non-null object
결제방법       37372 non-null int64
단위         22066 non-null object
브랜드        33066 non-null object
옵션         4140 non-null object
종류         22744 non-null object
동일상품시간차    36008 non-null timedelta64[ns]
dtypes: datetime64[ns](1), float64(2), int64(4), object(7), timedelta64[ns](1)
memory usage: 4.3+ MB


## 횟수 

### 3. 동일 상품 / 브랜드 총 판매횟수 
* NEW상품명 기준 ! 

In [124]:
# 동일 상품 총 판매횟수 
sales["상품총판매횟수"] = sales.groupby(["NEW상품명"])["방송일시"].transform('size')

In [ ]:
# 동일 브랜드 총 판매횟수 / 방송횟수 

# 판매횟수만 따지기 위해서는 원래 데이터를 봐야 하고, 
# 방송횟수를 따지기 위해서는 원래 데이터에서 '노출(분)' NaN값을 drop한 dataframe을 봐야 함 ! 

In [125]:
# 동일 브랜드 총 판매횟수 
sales["브랜드총판매횟수"] = sales.groupby(["브랜드"])["방송일시"].transform('size')

In [126]:
# 동일 브랜드 총 방송횟수 
sales3 = sales3.dropna(subset = ["노출(분)"]) # 20588 rows 
sales3["브랜드총방송횟수"] = sales3.groupby(["브랜드"])["방송일시"].transform('size')

In [127]:
sales.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,동일상품시간차,상품총판매횟수,브랜드총판매횟수
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,NaT,5,121.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,NaT,5,121.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0


In [133]:
sales3.head(3)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,동일상품시간차,브랜드총방송횟수
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,NaT,62.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,62.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,62.0


In [206]:
print(sales.shape)
sales.head()

(37372, 17)


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,동일상품시간차,상품총판매횟수,브랜드총판매횟수
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,NaT,5,121.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,NaT,5,121.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0


In [207]:
print(sales3.shape)
sales3.head()

(20588, 16)


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,동일상품시간차,브랜드총방송횟수
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,NaT,62.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,62.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,62.0
6,2019-01-01 07:00:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,6819000.0,오모떼 여성 레이스 파운데이션 브라,0,NaN,오모떼,NaN,4,NaT,272.0
7,2019-01-01 07:20:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,15689000.0,오모떼 여성 레이스 파운데이션 브라,0,NaN,오모떼,NaN,4,00:20:00,272.0


In [209]:
pd.merge(sales, sales3[["브랜드", '브랜드총방송횟수']].drop_duplicates() ,on = "브랜드", how="left") # 천재다...... 와 대박  짱ㅇ

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,동일상품시간차,상품총판매횟수,브랜드총판매횟수,브랜드총방송횟수
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,NaT,5,121.0,62.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,NaT,5,121.0,62.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0,62.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0,62.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,쿠첸 압력밥솥 6인용,1,6인용,쿠첸,NaN,NaN,00:00:00,500,1326.0,336.0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,쿠첸 압력밥솥 10인용,2,10인용,쿠첸,NaN,NaN,00:20:00,68,1326.0,336.0
37369,2020-01-01 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,쿠첸 압력밥솥 10인용,1,10인용,쿠첸,NaN,NaN,00:00:00,68,1326.0,336.0
37370,2020-01-01 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,쿠첸 압력밥솥 6인용,2,6인용,쿠첸,NaN,NaN,00:20:00,500,1326.0,336.0


In [160]:
# NaN값 없어지게 어케 합쳐...???
pd.merge(sales, sales3, how="outer").head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,동일상품시간차,상품총판매횟수,브랜드총판매횟수,브랜드총방송횟수
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,NaT,5,121.0,62.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,NaT,5,121.0,NaN
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0,62.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트 여성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0,NaN
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,5,121.0,62.0


* 누적판매량/누적주문량/누적방송횟수(카테고리(~ 1개월전,~ 2개월전,~ 6개월전, 없으면 None,&연속) : np.cumsum()   
* 조기매진 시간
* 결합상품(0 : 3+4 ,1 : 다른상품이 묶이면 or +) 
* 추세
* 가격별 카테고리 (6,5000미만 : 저가, 6,5000~20만원 : 중저가 , 20만 이상 : 고가) → 50000빼고 다시보기
* 주문량
* 할인률

## 매진
* 각 상품의 조기매진 여부 
* 조기매진 노출시간 (몇 분 남기고 조기매진 되었는지 ? )
* 조기매진 횟수 

In [161]:
sales.shape

(37372, 17)

In [ ]:
일단 onair 숫자만 가지고 체크를 했는데, 
해당 방송이 다음 방송과의 시간 차이를 계산하고 (dropna 해서 NaN값 없는 행들과 비교해야 할듯) 
onair 변수와 그 시간차이가 일치하는지, 일치하지 않는지로 비교해 봐야 할 것 같음 

In [192]:
soldout = sales3.copy()

In [193]:
soldout["다음방송시간"] = soldout["방송일시"].shift(-1) 

In [194]:
soldout["다음방송과의시간차"] = soldout["다음방송시간"] - soldout["방송일시"]

In [195]:
soldout["노출(분)"] = soldout["노출(분)"].apply(lambda x : datetime.timedelta(minutes=x)) # '분' 단위로 되어 있어서 바꿔주기

In [ ]:
soldout["다음방송과의시간차"] = soldout["다음방송과의시간차"].fillna(0) # 2020-1-1 00:00 상품값 0으로 채워줌 

In [196]:
soldout.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20588 entries, 0 to 37368
Data columns (total 18 columns):
방송일시         20588 non-null datetime64[ns]
노출(분)        20588 non-null timedelta64[ns]
마더코드         20588 non-null int64
상품코드         20588 non-null int64
상품명          20588 non-null object
상품군          20588 non-null object
판매단가         20588 non-null int64
취급액          20588 non-null float64
NEW상품명       20588 non-null object
결제방법         20588 non-null int64
단위           10923 non-null object
브랜드          16577 non-null object
옵션           2678 non-null object
종류           13533 non-null object
동일상품시간차      19657 non-null timedelta64[ns]
브랜드총방송횟수     16577 non-null float64
다음방송시간       20587 non-null datetime64[ns]
다음방송과의시간차    20587 non-null timedelta64[ns]
dtypes: datetime64[ns](2), float64(2), int64(4), object(7), timedelta64[ns](3)
memory usage: 3.0+ MB


In [211]:
soldout.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,동일상품시간차,브랜드총방송횟수,다음방송시간,다음방송과의시간차
0,2019-01-01 06:00:00,00:20:00,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,NaT,62.0,2019-01-01 06:20:00,00:20:00
2,2019-01-01 06:20:00,00:20:00,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,62.0,2019-01-01 06:40:00,00:20:00
4,2019-01-01 06:40:00,00:20:00,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,62.0,2019-01-01 07:00:00,00:20:00
6,2019-01-01 07:00:00,00:20:00,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,6819000.0,오모떼 여성 레이스 파운데이션 브라,0,NaN,오모떼,NaN,4,NaT,272.0,2019-01-01 07:20:00,00:20:00
7,2019-01-01 07:20:00,00:20:00,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,15689000.0,오모떼 여성 레이스 파운데이션 브라,0,NaN,오모떼,NaN,4,00:20:00,272.0,2019-01-01 07:40:00,00:20:00


In [202]:
soldout.loc[soldout["노출(분)"] != soldout["다음방송과의시간차"]]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,동일상품시간차,브랜드총방송횟수,다음방송시간,다음방송과의시간차
19,2019-01-01 11:20:00,00:20:00,100664,201999,크로커다일 The 편안한 코튼 브라팬티 4세트,속옷,69900,49829000.0,크로커다일 여성 The 편안한 코튼 브라팬티 4세트,0,4세트,크로커다일,NaN,4,00:20:00,77.0,2019-01-01 12:00:00,00:40:00
66,2019-01-02 01:40:00,00:17:06,100193,200652,레이프릴 무빙 맥시풀커버 브라팬티,속옷,69900,18695000.0,레이프릴 여성 무빙 맥시풀커버 브라팬티,0,NaN,레이프릴,NaN,4,00:20:00,206.0,2019-01-02 06:00:00,04:20:00
166,2019-01-03 00:20:00,00:20:00,100346,201072,테이트 남성 셀린니트3종,의류,39900,10481000.0,테이트 남성 셀린니트 3종,0,3종,테이트,NaN,0,00:20:00,62.0,2019-01-03 01:00:00,00:40:00
170,2019-01-03 01:40:00,00:17:06,100613,201894,푸마 웜셀 기모 치마레깅스 3종,속옷,59000,24910000.0,푸마 여성 웜셀 기모 치마레깅스 3종,0,3종,푸마,NaN,4,00:20:00,565.0,2019-01-03 06:00:00,04:20:00
265,2019-01-04 01:40:00,00:17:06,100229,200800,벨라홈 논슬립 옷걸이 세트,생활용품,29800,20974000.0,벨라홈 논슬립 옷걸이 세트,0,NaN,벨라홈,NaN,NaN,00:20:00,60.0,2019-01-04 06:00:00,04:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37280,2019-12-31 01:40:00,00:17:10,100439,201357,루시헨느 레이스 홀리데이 란쥬 패키지,속옷,89900,22453000.0,루시헨느 여성 레이스 홀리데이 란쥬 패키지,0,NaN,루시헨느,NaN,4,00:20:00,6.0,2019-12-31 02:00:00,00:20:00
37281,2019-12-31 02:00:00,00:20:00,100439,201357,루시헨느 레이스 홀리데이 란쥬 패키지,속옷,89900,29452000.0,루시헨느 여성 레이스 홀리데이 란쥬 패키지,0,NaN,루시헨느,NaN,4,00:20:00,6.0,2019-12-31 06:20:00,04:20:00
37311,2019-12-31 13:00:00,00:20:00,100837,202467,무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS),주방,218000,27909000.0,쿠쿠 전기밥솥 10인용,2,10인용,쿠쿠,NaN,NaN,00:20:00,416.0,2019-12-31 14:20:00,01:20:00
37358,2019-12-31 22:00:00,00:20:00,100155,200556,일시불 LG전자 매직스페이스 냉장고,가전,1359000,72733000.0,LG 매직스페이스 냉장고,1,NaN,LG,NaN,냉장고,00:20:00,1207.0,2019-12-31 23:20:00,01:20:00


In [200]:
sales.iloc[37358:37365] # 무형을 날리면 안되네..... 

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품명,결제방법,단위,브랜드,옵션,종류,동일상품시간차,상품총판매횟수,브랜드총판매횟수
37358,2019-12-31 22:00:00,20.0,100155,200556,일시불 LG전자 매직스페이스 냉장고,가전,1359000,72733000.0,LG 매직스페이스 냉장고,1,NaN,LG,NaN,냉장고,0 days 00:20:00,616,4565.0
37359,2019-12-31 22:00:00,NaN,100155,200489,무이자 LG전자 매직스페이스 냉장고,가전,1499000,32184000.0,LG 매직스페이스 냉장고,2,NaN,LG,NaN,냉장고,0 days 00:00:00,616,4565.0
37360,2019-12-31 23:20:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,4573000.0,쿠첸 압력밥솥 10인용,2,10인용,쿠첸,NaN,NaN,3 days 02:20:00,68,1326.0
37361,2019-12-31 23:20:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,9129000.0,쿠첸 압력밥솥 10인용,1,10인용,쿠첸,NaN,NaN,0 days 00:00:00,68,1326.0
37362,2019-12-31 23:20:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,620000.0,쿠첸 압력밥솥 6인용,2,6인용,쿠첸,NaN,NaN,3 days 02:20:00,500,1326.0
37363,2019-12-31 23:20:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,1664000.0,쿠첸 압력밥솥 6인용,1,6인용,쿠첸,NaN,NaN,0 days 00:00:00,500,1326.0
37364,2019-12-31 23:40:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,9149000.0,쿠첸 압력밥솥 10인용,2,10인용,쿠첸,NaN,NaN,0 days 00:20:00,68,1326.0
